Insert Data in Databse and start unification from Keys etc.

Manage Imports

In [ ]:
import os
import h5py
from pymongo import MongoClient
from dotenv import load_dotenv

Get Database URI from .env File and setup other variables. Be sure tu insert the DB URI for a MongoDB Database

In [ ]:
load_dotenv()

uri = os.environ['DB_URI']

# Verzeichnis, in dem sich die .h5 Dateien befinden
directory = './data'

# MongoDB Datenbank-Verbindung
client = MongoClient(uri)
db = client['rosen']
collection = db['big_data']


In [ ]:
def traverse_group(group, path=''):
    data = {}
    for key, item in group.items():
        new_key = key.lower().replace("_", "")  # Schlüssel in Kleinbuchstaben umwandeln und "_" entfernen
        if isinstance(item, h5py.Dataset):
            data[new_key] = item[()].tolist()  # Konvertieren in eine Liste für MongoDB
        elif isinstance(item, h5py.Group):  # Prüfen, ob es sich um eine Gruppe handelt
            data[new_key] = traverse_group(item, path + '/' + new_key)  # Rekursiver Aufruf
    return data


In [ ]:
def process_h5_file(file_path):
    with h5py.File(file_path, 'r') as h5file:
        data = traverse_group(h5file)  # Start der Rekursion von der Wurzelgruppe
        collection.insert_one(data)  # Daten in MongoDB einfügen
        print(f'Daten aus {file_path} wurden erfolgreich in MongoDB gespeichert.')

In [ ]:
# Durchlaufe alle .h5 Dateien im angegebenen Verzeichnis
for filename in os.listdir(directory):
    if filename.endswith('.h5'):
        process_h5_file(os.path.join(directory, filename))

print('Verarbeitung abgeschlossen.')

Refactor Document Structure. In den Dateien existieren die keys "Daten" und "Data" FML

In [ ]:
def transform_data(data):
    # Ermittle die Länge der Arrays und verwende die minimale Länge, um Indexfehler zu vermeiden
    min_length = min(len(v) for v in data.values() if isinstance(v, list))

    # Erstelle `measuring_points` basierend auf der minimalen Länge
    measuring_points = []
    for i in range(min_length):
        point = {key: data[key][i] if i < len(data[key]) else None for key in data if isinstance(data[key], list)}
        measuring_points.append(point)

    return measuring_points


def update_documents():
    for doc in collection.find():
        if 'data' in doc:
            # Transformiere die Daten
            new_measuring_points = transform_data(doc['data'])
            # Aktualisiere das Dokument mit den transformierten Daten unter dem neuen Schlüssel `measuring_points`
            collection.update_one({'_id': doc['_id']}, {'$set': {'measuring_points': new_measuring_points}})
            print(f'Dokument {doc["_id"]} wurde aktualisiert.')
        if 'daten' in doc:
            new_measuring_points = transform_data(doc['daten'])
            collection.update_one({'_id': doc['_id']}, {'$set': {'measuring_points': new_measuring_points}})
            print(f'Dokument {doc["_id"]} wurde aktualisiert.')


update_documents()
print('Alle Dokumente wurden aktualisiert.')


Unset unused (original) data

In [ ]:
collection.update_many({}, {"$unset": {"daten": "", "data": ""}})